In [ ]:
class Email:
    def __init__(self, sender, receiver, message):
        self.sender = sender
        self.receiver = receiver
        self.message = message
    
class NoEmailError(Exception):
    pass

def try_receive_email():
    # Email 인ㅅ턴스를 하나 반환하거나, NoEmailError를 발생시킨다.
    ...
    
def produce_emails(queue):
    while True:
        try:
            email = try_receive_email()
        except NoEmailError:
            return
        else:
            queue.append(email)
            
def consume_one_email(queue):
    if not queue:
        return
    
    email = queue.pop(0) # 소비자
    
    # 장기보관을 위해 메시지 인덱싱
    ...
    

def loop(queue, keep_running):
    while keep_running():
        produce_emails(queue)
        consume_one_email(queue)
        
def my_end_func():
    ...
    
loop([], my_end_func)

`produce_emails` 안에서 `try_receive_email`로 부터 전자우편을 받는데

Email을 `produce_emails` 안에서 처리하지 않는이유

종단 지연시간을 희생함으로써 안정적인 성능 프로파일과, 일관성있는 스루풋을 달성 가능하다.

이와 같은 생산자-소비자 큐로 리스트를 사용해도 어느정도 코드가 잘 동작한다.

하지만 크기가 늘어나면 리스트 타입의 성능은 선형보다 더 나빠진다.

다음은 List를 FIFO 큐로 사용할때 성능이 어떤지 timeit 내장 모듈을 통해 벤치마크를 수행한 결과이다.

In [5]:
import timeit

def print_results(count, tests):
    avg_iteration = sum(tests) / len(tests)
    print(f'\n 원소 수: {count:>5}, 걸린 시간: {avg_iteration:.6f}초')
    return count, avg_iteration

def list_append_benchmark(count):
    def run(queue):
        for i in range(count):
            queue.append(i)
            
            
    tests = timeit.repeat(
        setup='queue = []',
        stmt = 'run(queue)',
        globals = locals(),
        repeat = 1000,
        number = 1
    )
    
    return print_results(count, tests)


def print_delta(before, after):
    before_count, before_time = before
    after_count, after_time = after
    growth = 1 + (after_count - before_count) / before_count
    slowdown = 1 + (after_time - before_time) / before_time
    print(f'데이터 크기 {growth:>4.1f}배, 걸린 시간 {slowdown:>4.1f}배')

In [6]:
baseline = list_append_benchmark(500)
for count in (1000, 2000, 3000, 4000, 5000):
    comparison = list_append_benchmark(count)
    print_delta(baseline, comparison)


 원소 수:   500, 걸린 시간: 0.000126초

 원소 수:  1000, 걸린 시간: 0.000226초
데이터 크기  2.0배, 걸린 시간  1.8배

 원소 수:  2000, 걸린 시간: 0.000450초
데이터 크기  4.0배, 걸린 시간  3.6배

 원소 수:  3000, 걸린 시간: 0.000673초
데이터 크기  6.0배, 걸린 시간  5.3배

 원소 수:  4000, 걸린 시간: 0.000891초
데이터 크기  8.0배, 걸린 시간  7.0배

 원소 수:  5000, 걸린 시간: 0.001114초
데이터 크기 10.0배, 걸린 시간  8.8배


이 결과는 리스트 타입에 있는 append 메서드가 거의 상수 시간이 걸린다는 것을 보여준다.

데이터 크기가 커짐에 따라 큐에 데이터를 넣는데 걸리는 시간이 선형적으로 늘어난다.

다음 코드는 큐의 맨 앞에서 원소를 제거하는 pop(0) 호출 (소비자 함수의 리스트 사용법과 같다)를 벤치마크한다


In [7]:
def list_pop_benchmark(count):
    def prepare():
        return list(range(count))
    
    def run(queue):
        while queue:
            queue.pop(0)
            
    tests = timeit.repeat(
        setup = 'queue = prepare()',
        stmt = 'run(queue)',
        globals = locals(),
        repeat=1000,
        number=1
    )
    
    return print_results(count, tests)

In [9]:
baseline = list_pop_benchmark(500)

for count in (1000, 2000, 3000, 4000, 5000,6000):
    comparison = list_pop_benchmark(count)
    print_delta(baseline, comparison)
    
    


 원소 수:   500, 걸린 시간: 0.000167초

 원소 수:  1000, 걸린 시간: 0.000335초
데이터 크기  2.0배, 걸린 시간  2.0배

 원소 수:  2000, 걸린 시간: 0.000740초
데이터 크기  4.0배, 걸린 시간  4.4배

 원소 수:  3000, 걸린 시간: 0.001224초
데이터 크기  6.0배, 걸린 시간  7.3배

 원소 수:  4000, 걸린 시간: 0.001900초
데이터 크기  8.0배, 걸린 시간 11.4배

 원소 수:  5000, 걸린 시간: 0.003002초
데이터 크기 10.0배, 걸린 시간 17.9배

 원소 수:  6000, 걸린 시간: 0.004329초
데이터 크기 12.0배, 걸린 시간 25.9배


큐 길이가 늘어남에 따라 큐 길이의 제곱에 비례해 늘어난다

pop(0)을 하면 리스트의 남은 모든 원소를 제 위치로 옮겨야 해서 

결과적으로 전체 리스트 내용을 다시 재대입 하기 때문이다. 

리스트의 모든 원소에 대해 pop(0)을 호출하므로 len(queue) * len(queue)의 연산을 수행해야 모든 대기열 원소를 소비 할 수 있다.

파이썬 collections 내장 모듈에는 deque 클래스가 들어있다

deque는 양방향 큐 구현이며, 데크 라고 읽는다. 

데크의 시작과 끝 지점에 원소를 넣거나 빼는데는 상수 시간이 걸린다. 따라서 데크는 FIFO queue를 구현할 떄 이상적이다.

deque를 사용하더라도 produce_emails 함수에 있는 append는 리스트를 사용할 때와 같은 형태로 그대로 둬도 된다.

하지만 consume_one_email에 있는 list.pop 메서드는 deque.popleft 메서드를 아무 인자도 없이 호출하게 바꿔야한다

loop 메서드 호출할때 list instance 대신 deque 인스턴스를 전달해야한다